In [1]:
from chain_builder import get_sentence
import cPickle as pickle
import tweepy
from datetime import date, timedelta
from keys import keys

In [2]:
with open('Pickles/forward_dict.pkl') as f:
    f_dict = pickle.load(f)

with open('Pickles/reversed_dict.pkl') as f:
    r_dict = pickle.load(f)

In [18]:
CONSUMER_KEY = keys['consumer_key']
CONSUMER_SECRET = keys['consumer_secret']
ACCESS_TOKEN = keys['access_token']
ACCESS_TOKEN_SECRET = keys['access_token_secret']

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweet_replies = []

today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)

# ******************************************************
# Queries Popular Tweets directed at Candidates or using
# well-known Hashtags and responds to all queried tweets.
# ******************************************************
hashtags = ["#FeelTheBern OR #ImWithHer OR #MakeAmericaGreatAgain OR #Trump2016 \
OR @BernieSanders OR @HillaryClinton OR @realDonaldTrump"]

top_tweets = tweepy.Cursor(api.search, q=hashtags, count=100, result_type='popular', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in top_tweets:
    if status.favorite_count > 1000:
        tweet_replies.append(status)

# ***************************************************
# Queries Tweets directed at or in reply to
# FeelTheBern_Bot and responds to all queried tweets.
# ***************************************************
bernbot_query = "to:FeelTheBern_Bot OR @FeelTheBern_Bot"

bernbot_tweets = tweepy.Cursor(api.search, q=bernbot_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in bernbot_tweets:
    tweet_replies.append(status)

# ************************************************
# Queries Tweets from Presidential Candidates and
# responds to those with greater than 500 likes.
# ************************************************
candidate_query = "from:BernieSanders OR from:HillaryClinton OR from:realDonaldTrump"

candidate_tweets = tweepy.Cursor(api.search, q=candidate_query, count=100, result_type='recent', include_entities=True, monitor_rate_limit=True, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, lang="en", since=yesterday, until=today).items()

for status in candidate_tweets:
    if status.favorite_count > 500:
        tweet_replies.append(status)


# DROPS ANY DUPLICATE TWEETS SO IT DOES NOT RESPOND TWICE
tweet_replies = set(tweet_replies)

In [20]:
best_tweets = []
for i, tweet in enumerate(tweet_replies):
    resp_length = 141
    text = tweet.text
    sn = tweet.user.screen_name
    response = ':'

    while resp_length > 140 or ':' in response:
        response = get_sentence(text, f_dict, r_dict, randomness = 1) 
        response = "@{0} {1}".format(sn, response)
        resp_length = len(response)
    
    print i, '----', tweet.id
    print '@', sn, text
    print response
    print '*************'
    best_tweets.append([response, tweet.id])

0 ---- 727281328622804992
@ DanScavino #Trump2016 #BTS- Indianapolis, Indiana-- prior to departing for South Bend- thanking all law enforcement officers. https://t.co/s1cUPjfbPm
@DanScavino Now we want to welcome Sebastian de la Cruz (ph) who was Justice Scalia's first law clerk.
*************
1 ---- 727176416345862145
@ DanScavino Crooked Hillary has bad judgement. 
#Trump2016 #TrumpTrain🚂💨🇺🇸🇺🇸 https://t.co/QikxtMy7I5
@DanScavino But, you're calling them hillary Clinton's failed trade policies.
*************
2 ---- 726935349973385216
@ HillaryClinton We should be making it easier to vote, not harder. The attacks on voting rights have to end.
@HillaryClinton The federal government wants Apple to unlock the phone used by that San Bernardino terrorist to prevent future attacks.
*************
3 ---- 727169840776118272
@ realDonaldTrump I will defeat Crooked Hillary Clinton on 11/8/2016. #Trump2016
#MakeAmericaGreatAgain https://t.co/HtwD1FGn9e
@realDonaldTrump So he had to make that case 

In [26]:
best_list = [16]
for i, tweet in enumerate(best_tweets):
    if i in best_list:
        print i 
        print tweet[0], tweet[1]
        s = api.update_status(tweet[0], tweet[1])

16
@BernieSanders Countries we are defending must pay for the wall? 726949851355734016
